In [1]:
from Crypto.Util.number import inverse
class Field:
    def __init__(self, p: int):
        self.p = p
    def add(self, a: int, b: int):
        return (a+b)%self.p
    def mul(self, a: int, b: int):
        return (a*b)%self.p
    def inv(self, a: int):
        return inverse(a, self.p)
f = Field(11)
f.add(5, 8)

2

In [ ]:
E = EllipticCurve(5, 2, 1)
for i in range(1,10):
    temp = ec_mul(E, Point(0,1,False), i)
    print(temp.x, temp.y)
    print(temp.x**5 % 5, temp.y**5 % 5)

In [47]:

from math import inf, prod
from typing import List
from sympy import factorint, isprime, prime, integer_nthroot, mod_inverse
from sympy.polys.polytools import invert
from sympy import symbols, Poly, div

from collections import namedtuple
from Crypto.Util.number import inverse

Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return "Point("+str(p.x)+","+str(p.y)+")" if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "p a b")

def ec_frobenius(curve: EllipticCurve, P: Point):
    """Apply the Frobenius map to point P = (x, y) on the curve."""
    return Point(pow(P.x, curve.p, curve.p), pow(P.y, curve.p, curve.p), P.isinf)

def ec_neg(curve: EllipticCurve, p: Point):
    return Point(p.x, -p.y % curve.p, p.isinf)

def ec_add(curve: EllipticCurve, p1: Point, p2: Point):
    assert not (p1.isinf and p2.isinf), "cannot add O to O"
    if p1.isinf: return p2
    if p2.isinf: return p1
    if (p1.x==p2.x and p1.y==-p2.y): return Point(0,0,isinf=True)
    if p1.x!=p2.x:
        # print(p2.x, p1.x, curve.p)
        tangent = (p2.y - p1.y)*inverse(p2.x - p1.x, curve.p) # pow(p2.x - p1.x, curve.p-2, curve.p)
    else:
        tangent = (3*(p1.x)**2 + curve.a)*inverse(2*p1.y, curve.p) # pow(2*p1.y, curve.p-2, curve.p)
    p3x = tangent**2 - p1.x - p2.x
    p3y = tangent*(p1.x - p3x) - p1.y
    return Point(p3x%curve.p, p3y%curve.p)

def ec_mul(curve: EllipticCurve, p: Point, n: int):
    if n==0: return Point(p.x, p.y, True)
    p1 = p
    p2 = Point(0,0,True)
    while (n>0):
        if n%2==1: p2 = ec_add(curve, p1, p2)
        p1 = ec_add(curve, p1, p1)
        n = n//2
    return p2

division_polynomial_cache = {}
x, y = symbols('x y')
def reduce_mod_p(poly, p):
    """Reduce a polynomial modulo p, but leave y terms untouched."""
    # Get the coefficients of the polynomial
    coeffs = poly.all_coeffs()
    # Reduce only numeric coefficients modulo p
    mod_coeffs = [c % p if c.is_number else c for c in coeffs]
    # Return a new polynomial with reduced coefficients
    return Poly.from_list(mod_coeffs, gens=poly.gens)


# Define the division polynomial function with modular arithmetic
def division_polynomial(E: EllipticCurve, n):
    """Calculate the n-th division polynomial modulo p for the elliptic curve y^2 = x^3 + ax + b."""
    a, b, p = E.a, E.b, E.p
    DIVPOL_BASE_CASE = [
        Poly(0, x),
        Poly(1, x),
        Poly(2 * y, x),
        Poly(3 * x**4 + 6 * a * x**2 + 12 * b * x - a**2, x),
        Poly(4 * y * (x**6 + 5 * a * x**4 + 20 * b * x**3 - 5 * a**2 * x**2 - 4 * a * b * x - 8 * b**2 - a**3), x)
    ]
    # print(DIVPOL_BASE_CASE[4]*DIVPOL_BASE_CASE[2]**3 - DIVPOL_BASE_CASE[1]*DIVPOL_BASE_CASE[3]**3)
    if n in division_polynomial_cache:
        return division_polynomial_cache[n]
    if n <= 4:
        result = DIVPOL_BASE_CASE[n]
    elif n % 2 == 1:  # Odd n: psi_2k+1
        m = (n - 1) // 2
        psi_m = division_polynomial(E, m)
        psi_m1 = division_polynomial(E, m + 1)
        psi_m2 = division_polynomial(E, m + 2)
        psi_ms1 = division_polynomial(E, m - 1)
        result = psi_m2 * psi_m**3 - psi_ms1 * psi_m1**3
        # result = reduce_mod_p(result, p)  # Apply mod p reduction
    else:  # Even n: psi_2k
        m = n // 2
        psi_m = division_polynomial(E, m)
        psi_m1 = division_polynomial(E, m + 1)
        psi_m2 = division_polynomial(E, m + 2)
        psi_ms1 = division_polynomial(E, m - 1)
        psi_ms2 = division_polynomial(E, m - 2)
        result = (psi_m*invert(2 * y, p))*(psi_m2*psi_ms1**2 - psi_ms2*psi_m1**2)
        # result = reduce_mod_p(result, p)  # Apply mod p reduction
    division_polynomial_cache[n] = result
    return result
from functools import cache

def division_polynomial(a, b, x, n):
    F = 4*(x**3 + a*x + b)

    @cache
    def f(n):
        if n == 0:
            return 0
        elif n == 1:
            return 1
        elif n == 2:
            return 1
        elif n == 3:
            return 3*x**4 + 6*a*x**2 + 12*b*x - a**2
        elif n == 4:
            return 2*(x**6 + 5*a*x**4 + 20*b*x**3 - 5*a**2*x**2 - 4*a*b*x - 8*b**2 - a**3)
        elif n % 2 == 0:
            m = n//2
            return (f(m + 2) * f(m - 1)**2 - f(m-2)*f(m+1)**2) * f(m)
        elif n % 4 == 1:
            m = (n - 1) // 2
            return F**2*f(m+2)*f(m)**3 - f(m-1)*f(m+1)**3
        elif n % 4 == 3:
            m = (n - 1) // 2
            return f(m+2)*f(m)**3 - F**2*f(m-1)*f(m+1)**3
        else:
            assert False, "unreachable"
    return f(n)

In [60]:
2 * 2*(x**3 + 1*x + 1)

4*x**3 + 4*x + 4

In [59]:
from sympy import primerange, sqrt, simplify
E = EllipticCurve(101, 1, 1)
simplify(division_polynomial(E, 4).subs(y, x**3+x+1))
division_polynomial(E, 4)

Poly(4*y*x**6 + 20*y*x**4 + 80*y*x**3 - 20*y*x**2 - 16*y*x - 36*y, x, domain='ZZ[y]')

In [19]:
primes = list(primerange(6))
for l in primes:
    psi_l = division_polynomial(E, l)
    print(l, psi_l)

Poly(-27*x**12 - 324*x**10 - 972*x**9 - 1188*x**8 - 7776*x**7 + (32*y**4 - 12528)*x**6 - 12960*x**5 + (320*y**4 - 45072)*x**4 + (1920*y**4 - 36288)*x**3 + (14976 - 640*y**4)*x**2 + (-768*y**4 - 1728)*x + 64 - 2560*y**4, x, domain='ZZ[y]')
2 Poly(2*y, x, domain='ZZ[y]')
Poly(-27*x**12 - 324*x**10 - 972*x**9 - 1188*x**8 - 7776*x**7 + (32*y**4 - 12528)*x**6 - 12960*x**5 + (320*y**4 - 45072)*x**4 + (1920*y**4 - 36288)*x**3 + (14976 - 640*y**4)*x**2 + (-768*y**4 - 1728)*x + 64 - 2560*y**4, x, domain='ZZ[y]')
3 Poly(3*x**4 + 12*x**2 + 36*x - 4, x, domain='ZZ')
Poly(-27*x**12 - 324*x**10 - 972*x**9 - 1188*x**8 - 7776*x**7 + (32*y**4 - 12528)*x**6 - 12960*x**5 + (320*y**4 - 45072)*x**4 + (1920*y**4 - 36288)*x**3 + (14976 - 640*y**4)*x**2 + (-768*y**4 - 1728)*x + 64 - 2560*y**4, x, domain='ZZ[y]')
Poly(-27*x**12 - 324*x**10 - 972*x**9 - 1188*x**8 - 7776*x**7 + (32*y**4 - 12528)*x**6 - 12960*x**5 + (320*y**4 - 45072)*x**4 + (1920*y**4 - 36288)*x**3 + (14976 - 640*y**4)*x**2 + (-768*y**4 - 1728)*

In [78]:

print("Psi_4 mod p:", division_polynomial(E, 4))
print("Psi_5 mod p:", division_polynomial(E, 5))

Psi_3 mod p: Poly(3*x**4 + 12*x**2 + 36*x + 97, x, domain='ZZ')
Psi_4 mod p: Poly(4*y*x**6 + 40*y*x**4 + 240*y*x**3 - 80*y*x**2 - 96*y*x - 320*y, x, domain='ZZ[y]')
Psi_5 mod p: Poly(74*x**12 + 80*x**10 + 38*x**9 + 24*x**8 + x**7 + (32*y**4 - 34344)*x**6 + 69*x**5 + (320*y**4 - 173241)*x**4 + (1920*y**4 - 298080)*x**3 + (-640*y**4 - 715860)*x**2 + (-768*y**4 - 1016172)*x - 2560*y**4 - 912673, x, domain='ZZ[y]')


In [41]:
class EllipticCurve:
    def __init__(self, a, b, p):
        self.a = a  # coefficient of x
        self.b = b  # constant term
        self.p = p  # prime modulus (field characteristic)
        
    def mod_inv(self, n):
        """Compute the modular inverse of n modulo p using the extended Euclidean algorithm."""
        return pow(n, self.p - 2, self.p)
    
    def add_points(self, P, Q):
        """Add two points P and Q on the elliptic curve."""
        if P == "O":  # Identity element (point at infinity)
            return Q
        if Q == "O":
            return P
        
        x1, y1 = P
        x2, y2 = Q
        
        # Point doubling case
        if P == Q:
            if y1 == 0:
                return "O"  # The result is the point at infinity
            m = (3 * x1**2 + self.a) * self.mod_inv(2 * y1) % self.p
        # General addition case
        else:
            if x1 == x2:
                return "O"  # Points add to infinity if x1 == x2 and y1 != y2
            m = (y2 - y1) * self.mod_inv(x2 - x1) % self.p
        
        # Calculate x3 and y3
        x3 = (m**2 - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)
    def scalar_multiply(self, k, P):
        """Multiply a point P by an integer k using the double-and-add method."""
        result = "O"  # Start with the point at infinity
        addend = P

        while k > 0:
            if k & 1:  # If the current bit is 1, add the addend to the result
                result = self.add_points(result, addend)
            # Double the point for the next bit
            addend = self.add_points(addend, addend)
            k >>= 1  # Shift k to the right by 1 bit

        return result
    # you learnt this, frobenius is when x^p = x mod p
    def frobenius(self, P):
        """Apply the Frobenius map to point P = (x, y) on the curve."""
        if P == "O":
            return "O"
        
        x, y = P
        x_frobenius = pow(x, self.p, self.p)
        y_frobenius = pow(y, self.p, self.p)
        
        return (x_frobenius, y_frobenius)

In [42]:
class Polynomial:
    def __init__(self, coeffs):
        """Initialize a polynomial with coefficients given in a list."""
        self.coeffs = coeffs  # coeffs[i] corresponds to the x^i term
    
    def evaluate(self, x, p):
        """Evaluate the polynomial at x modulo p."""
        result = 0
        for i, coeff in enumerate(self.coeffs):
            result = (result + coeff * pow(x, i, p)) % p
        return result
    
    def reduce(self, p):
        """Reduce the polynomial coefficients modulo p."""
        return [c % p for c in self.coeffs]


In [43]:
def small_primes(bound):
    """Generate a list of small primes up to a given bound.
    Example: 2, 3, 5, 7, 11
    """
    primes = []
    for num in range(2, bound + 1):
        prime = True
        for i in range(2, int(num ** 0.5) + 1):
            if num % i == 0:
                prime = False
                break
        if prime:
            primes.append(num)
    return primes
def compute_t_mod_l(curve, P, l):
    """Compute t mod l by comparing Frobenius(P) and [t]P for small l.
    FOr some small primes, check if the frobenius endomorphism holds
    """
    for t in range(l):
        # Scalar multiply P by t mod l
        tP = curve.scalar_multiply(t, P)
        # Apply the Frobenius map to P
        frobP = curve.frobenius(P)
        # Check if Frobenius(P) equals [t]P mod l
        if frobP == tP:
            return t
    return None  # If no t works, return None
def chinese_remainder_theorem(remainders, moduli):
    """Solve the system of congruences using the Chinese Remainder Theorem.
    crt crt crt...
    """
    total = 0
    product = 1
    for mod in moduli:
        product *= mod  # Compute the product of all moduli
    
    for remainder, mod in zip(remainders, moduli):
        partial_product = product // mod
        inverse = pow(partial_product, -1, mod)  # Modular inverse of partial_product mod mod
        total += remainder * inverse * partial_product
    
    return total % product
def combine_modular_traces(curve, P, primes):
    """Compute t using small primes and the Chinese Remainder Theorem."""
    remainders = []
    for l in primes:
        t_mod_l = compute_t_mod_l(curve, P, l)
        if t_mod_l is None:
            raise ValueError(f"Could not find t mod {l}")
        remainders.append(t_mod_l)
    
    t = chinese_remainder_theorem(remainders, primes)
    return t
def compute_total_points(p, t):
    """Compute the total number of points N on the elliptic curve."""
    N = p + 1 - t
    return N

In [48]:
a=3
b=2
p=113
curve = EllipticCurve(a,b,p)

# Step 1: Generate small primes up to a certain bound
primes = small_primes(113)

# Step 2: Compute the combined trace t mod L using small primes
P = (0, 1)
t_combined = combine_modular_traces(curve, P, primes)

# Step 3: Compute the total number of points on the curve using the trace t
N = compute_total_points(p, t_combined)

# Print the result
print(f"Total number of points on the elliptic curve over F_{p}: N = {N}")

Total number of points on the elliptic curve over F_113: N = 113


In [40]:
# Define the elliptic curve
a=3
b=2
p=5
curve = EllipticCurve(a,b,p)

# Define points on the curve
P = (0, 1)
Q = (1, 2)

# Test combining modular traces
primes = small_primes(99)  # Use small primes up to 5
t_combined = combine_modular_traces(curve, P, primes)
print(f"Combined t = {t_combined}")

# Test the total points calculation
N = compute_total_points(p, t_combined)
print(f"Total number of points on the elliptic curve: N = {N}")

Combined t = 1
Total number of points on the elliptic curve: N = 5


In [32]:
[curve.scalar_multiply(i, P) for i in range(1,10)]

[(0, 1), (1, 3), (3, 3), (3, 2), (1, 2), (0, 4), 'O', (0, 1), (1, 3)]

In [16]:
# Create a polynomial f(x) = 2 + 3x + 4x^2
poly = Polynomial([2, 3, 4])

# Test polynomial evaluation at x = 1 modulo p = 5
result = poly.evaluate(1, 5)
print(f"f(1) mod 5 = {result}")

f(1) mod 5 = 4
